In [98]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [99]:
# Apriori 算法
# 1. 支持度计算
def loadDataSet():
        # 读取数据txt中的数据
        data = pd.read_csv('Transactions.csv')
        return data

In [100]:
def createC1(dataSet):# 创建候选项集
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))# 使用frozenset是为了后面可以将这些值作为字典的键

def scanD(D, Ck, minSupport):# 从候选项集中筛选出满足最小支持度的项集
    # D: 数据集 Ck: 候选项集  minSupport: 最小支持度
    ssCnt = {}

    for tid in D: # 遍历数据集中的每一条交易记录
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: # 如果不在字典中，就添加进去
                    ssCnt[can] = 1
                else: # 如果在字典中，就加1
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData # 返回满足最小支持度的项集和对应的支持度



In [128]:
# 2. 生成候选项集
def aprioriGen(Lk, k):# 生成候选项集
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

def apriori(dataSet, minSupport = 0.0): # 生成所有满足最小支持度的项集
    # dataSet: 数据集  minSupport: 最小支持度
    C1 = createC1(dataSet)
    # D = list(map(set, dataSet))
    D = []
    # 遍历每一行
    for index, row in dataSet.iterrows():
        # 创建一个空集合来存储这一行的列名
        s = set()
        # 遍历除了views之外的所有列
        for col in dataSet.columns[:-1]:
        # 如果这个数据是1，就把这列的名称加入到集合中
            if row[col] == 1:
                s.add(col)
        # 将s转为list
        s = list(s)
        # 把这一行对应的集合加入到列表中
        D.append(s)

    # print (D)
    L1, supportData = scanD(D, C1, minSupport)
    print (L1)
    L = [L1]
    k = 2 # k表示项集中元素的个数
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData # 返回所有满足最小支持度的项集和对应的支持度


In [ ]:
# 2. 生成候选项集
def aprioriGen(Lk, k):# 生成候选项集
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

def apriori(dataSet, minSupport = 0.05): # 生成所有满足最小支持度的项集
    # dataSet: 数据集  minSupport: 最小支持度
    C1 = createC1(dataSet)
    # D = list(map(set, dataSet))
    D = []
    # 遍历每一行
    for index, row in dataSet.iterrows():
        # 创建一个空集合来存储这一行的列名
        s = set()
        # 遍历除了views之外的所有列
        for col in dataSet.columns[:-1]:
        # 如果这个数据是1，就把这列的名称加入到集合中
            if row[col] == 1:
                s.add(col)
        # 将s转为list
        s = list(s)
        # 把这一行对应的集合加入到列表中
        D.append(s)

    print (D)
    L1, supportData = scanD(D, C1, minSupport)
    print (L1)
    L = [L1]
    k = 2 # k表示项集中元素的个数
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData # 返回所有满足最小支持度的项集和对应的支持度


In [164]:
# 3. 生成关联规则
def generateRules(L, supportData, minConf=0.1):# 生成关联规则
    bigRuleList = []
    
    for i in range(1, len(L)):
        for freqSet in L[i]: # 遍历频繁项集中的每一个项集

            print(freqSet)

            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1): # 如果频繁项集中的元素超过2个，就需要进一步合并
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

def calcConf(freqSet, H, supportData, brl, minConf=0.7):# 计算置信度
    # freqSet:频繁项集 H:频繁项集中所有的元素的集合  supportData:支持度数据  brl:关联规则列表  minConf:最小置信度
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq] # 计算置信度
        if conf >= minConf:
            print(freqSet-conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):# 生成候选规则集:
    # freqSet:频繁项集 H:频繁项集中所有的元素的集合  supportData:支持度数据  brl:关联规则列表  minConf:最小置信度
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

    

In [103]:
# test
dataSet = loadDataSet()
dataSet.head()

,COD,pasta,milk,water,biscuits,coffee,brioches,yoghurt,frozen vegetables,tunny,...,tomato souce,coke,rice,juices,crackers,oil,frozen fish,ice cream,mozzarella,tinned meat
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
1,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,4,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [104]:
dataSet.drop(['COD'], axis=1, inplace=True)
dataSet.head()

,pasta,milk,water,biscuits,coffee,brioches,yoghurt,frozen vegetables,tunny,beer,tomato souce,coke,rice,juices,crackers,oil,frozen fish,ice cream,mozzarella,tinned meat
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [105]:
# 将数据集转换为集合,如果对应项为0，则将这一列的名称不添加到集合中，如果为不为0，则添加到集合中
# 假设df是Pandas读入的数据
# 创建一个空列表来存储每一行对应的集合
D = []
# 遍历每一行
for index, row in dataSet.iterrows():
    # 创建一个空集合来存储这一行的列名
    s = set()
    # 遍历除了views之外的所有列
    for col in dataSet.columns[:-1]:
    # 如果这个数据是1，就把这列的名称加入到集合中
        if row[col] == 1:
            s.add(col)
    # 将s转为list
    s = list(s)
    # 把这一行对应的集合加入到列表中
    D.append(s)

In [106]:
D

[['tunny', 'mozzarella'],
 ['milk'],
 [' pasta ', 'milk', 'coke', 'crackers'],
 ['water'],
 ['tunny'],
 [' pasta ', 'milk', 'biscuits', 'crackers'],
 ['biscuits', 'milk', 'juices'],
 [' pasta '],
 ['oil'],
 ['biscuits'],
 [],
 [' pasta ', 'milk', 'coffee'],
 ['tunny', 'milk', 'tomato souce', 'juices', ' pasta '],
 ['milk'],
 [' pasta ', 'yoghurt'],
 ['biscuits', 'milk', 'crackers', 'brioches'],
 [' pasta '],
 [' pasta ', 'milk'],
 [' pasta ', 'juices', 'coffee'],
 ['juices', 'brioches'],
 ['biscuits'],
 [' pasta '],
 ['milk'],
 ['water', 'milk', 'coffee'],
 [' pasta '],
 ['coke', 'beer'],
 [' pasta ', 'milk'],
 ['biscuits', 'mozzarella'],
 ['frozen vegetables'],
 [' pasta ', 'milk'],
 ['rice'],
 [' pasta ', 'biscuits', 'crackers'],
 [' pasta ', 'milk', 'frozen vegetables'],
 ['milk', 'yoghurt', 'coffee'],
 ['water', 'rice', 'tomato souce'],
 ['milk'],
 ['biscuits'],
 ['tunny', 'milk', 'brioches'],
 [' pasta ', 'milk', 'tomato souce', 'crackers'],
 ['milk'],
 ['frozen vegetables'],
 ['b

In [107]:
C1 = createC1(D) # 创建候选项集
C1

[frozenset({' pasta '}),
 frozenset({'beer'}),
 frozenset({'biscuits'}),
 frozenset({'brioches'}),
 frozenset({'coffee'}),
 frozenset({'coke'}),
 frozenset({'crackers'}),
 frozenset({'frozen fish'}),
 frozenset({'frozen vegetables'}),
 frozenset({'ice cream'}),
 frozenset({'juices'}),
 frozenset({'milk'}),
 frozenset({'mozzarella'}),
 frozenset({'oil'}),
 frozenset({'rice'}),
 frozenset({'tomato souce'}),
 frozenset({'tunny'}),
 frozenset({'water'}),
 frozenset({'yoghurt'})]

In [108]:
L1, supportData0 = scanD(D, C1, 0.0) # 从候选项集中筛选出满足最小支持度的项集
print(L1) # 打印出满足最小支持度的项集
print(supportData0) # 打印出每个项集的支持度

[frozenset({'ice cream'}), frozenset({'frozen fish'}), frozenset({'rice'}), frozenset({'frozen vegetables'}), frozenset({'beer'}), frozenset({'brioches'}), frozenset({'yoghurt'}), frozenset({'tomato souce'}), frozenset({'coffee'}), frozenset({'oil'}), frozenset({'juices'}), frozenset({'biscuits'}), frozenset({'water'}), frozenset({'crackers'}), frozenset({'coke'}), frozenset({' pasta '}), frozenset({'milk'}), frozenset({'tunny'}), frozenset({'mozzarella'})]
{frozenset({'mozzarella'}): 0.01513742620504725, frozenset({'tunny'}): 0.10386436866120277, frozenset({'milk'}): 0.46132387604610425, frozenset({' pasta '}): 0.35034491706852927, frozenset({'coke'}): 0.10706485305884134, frozenset({'crackers'}): 0.030404601777566334, frozenset({'water'}): 0.27850701727829075, frozenset({'biscuits'}): 0.20474450187055337, frozenset({'juices'}): 0.08256384750124343, frozenset({'oil'}): 0.0356594511601756, frozenset({'coffee'}): 0.15027139242696191, frozenset({'tomato souce'}): 0.11597430962524058, fro

In [109]:
L, supportData = apriori(D) # 生成所有满足最小支持度的项集
print(L) # 打印出所有满足最小支持度的项集
print(supportData) # 打印出每个项集的支持度

[[]]
{frozenset({'mozzarella'}): 0.01513742620504725, frozenset({'tunny'}): 0.10386436866120277, frozenset({'milk'}): 0.46132387604610425, frozenset({' pasta '}): 0.35034491706852927, frozenset({'coke'}): 0.10706485305884134, frozenset({'crackers'}): 0.030404601777566334, frozenset({'water'}): 0.27850701727829075, frozenset({'biscuits'}): 0.20474450187055337, frozenset({'juices'}): 0.08256384750124343, frozenset({'oil'}): 0.0356594511601756, frozenset({'coffee'}): 0.15027139242696191, frozenset({'tomato souce'}): 0.11597430962524058, frozenset({'yoghurt'}): 0.15234738230651126, frozenset({'brioches'}): 0.15319075319507816, frozenset({'beer'}): 0.1092057176221266, frozenset({'frozen vegetables'}): 0.11389831974569124, frozenset({'rice'}): 0.06502605799796726, frozenset({'frozen fish'}): 0.025279501762428908, frozenset({'ice cream'}): 0.01656466924723742}


In [110]:
rules = generateRules(L, supportData, minConf=0.7) # 生成关联规则
print(rules) # 打印出关联规则

[]


In [111]:
# test 
dataSet = loadDataSet()
dataSet.head()

,COD,pasta,milk,water,biscuits,coffee,brioches,yoghurt,frozen vegetables,tunny,...,tomato souce,coke,rice,juices,crackers,oil,frozen fish,ice cream,mozzarella,tinned meat
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
1,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,4,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [112]:
dataSet.drop(['COD'], axis=1, inplace=True)
dataSet.head()

,pasta,milk,water,biscuits,coffee,brioches,yoghurt,frozen vegetables,tunny,beer,tomato souce,coke,rice,juices,crackers,oil,frozen fish,ice cream,mozzarella,tinned meat
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [174]:

# 将数据集转换为集合,如果对应项为0，则将这一列的名称不添加到集合中，如果为不为0，则添加到集合中
# 假设df是Pandas读入的数据
# 创建一个空列表来存储每一行对应的集合

D = []
# 遍历每一行
for index, row in dataSet.iterrows():
    # 创建一个空集合来存储这一行的列名
    s = []
    # 遍历除了views之外的所有列
    for col in dataSet.columns[:-1]:
    # 如果这个数据是1，就把这列的名称加入到集合中
        if row[col] == 1:
            s.append(col)
    # 把这一行对应的集合加入到列表中
    D.append(s)

D

[['tunny', 'mozzarella'],
 ['milk'],
 [' pasta ', 'milk', 'coke', 'crackers'],
 ['water'],
 ['tunny'],
 [' pasta ', 'milk', 'biscuits', 'crackers'],
 ['milk', 'biscuits', 'juices'],
 [' pasta '],
 ['oil'],
 ['biscuits'],
 [],
 [' pasta ', 'milk', 'coffee'],
 [' pasta ', 'milk', 'tunny', 'tomato souce', 'juices'],
 ['milk'],
 [' pasta ', 'yoghurt'],
 ['milk', 'biscuits', 'brioches', 'crackers'],
 [' pasta '],
 [' pasta ', 'milk'],
 [' pasta ', 'coffee', 'juices'],
 ['brioches', 'juices'],
 ['biscuits'],
 [' pasta '],
 ['milk'],
 ['milk', 'water', 'coffee'],
 [' pasta '],
 ['beer', 'coke'],
 [' pasta ', 'milk'],
 ['biscuits', 'mozzarella'],
 ['frozen vegetables'],
 [' pasta ', 'milk'],
 ['rice'],
 [' pasta ', 'biscuits', 'crackers'],
 [' pasta ', 'milk', 'frozen vegetables'],
 ['milk', 'coffee', 'yoghurt'],
 ['water', 'tomato souce', 'rice'],
 ['milk'],
 ['biscuits'],
 ['milk', 'brioches', 'tunny'],
 [' pasta ', 'milk', 'tomato souce', 'crackers'],
 ['milk'],
 ['frozen vegetables'],
 ['b

In [181]:
def apriori_test(dataSet, minSupport = 0.05):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [176]:
print(type(D))
print(type(D[0]))

<class 'list'>
<class 'list'>


In [184]:
L, supportData = apriori_test(D) # 生成所有满足最小支持度的项集
print(L) # 打印出所有满足最小支持度的项集
print(supportData) # 打印出每个项集的支持度

[[frozenset({'rice'}), frozenset({'frozen vegetables'}), frozenset({'beer'}), frozenset({'brioches'}), frozenset({'yoghurt'}), frozenset({'tomato souce'}), frozenset({'coffee'}), frozenset({'juices'}), frozenset({'biscuits'}), frozenset({'water'}), frozenset({'coke'}), frozenset({' pasta '}), frozenset({'milk'}), frozenset({'tunny'})], [frozenset({' pasta ', 'brioches'}), frozenset({'water', ' pasta '}), frozenset({'water', 'biscuits'}), frozenset({'milk', 'yoghurt'}), frozenset({'frozen vegetables', 'milk'}), frozenset({'water', 'milk'}), frozenset({'milk', 'brioches'}), frozenset({' pasta ', 'yoghurt'}), frozenset({'milk', 'tomato souce'}), frozenset({' pasta ', 'tomato souce'}), frozenset({'milk', 'coffee'}), frozenset({' pasta ', 'coffee'}), frozenset({'biscuits', 'milk'}), frozenset({'biscuits', ' pasta '}), frozenset({' pasta ', 'milk'}), frozenset({'coke', 'milk'})], [frozenset({'water', ' pasta ', 'milk'})], []]
{frozenset({'mozzarella'}): 0.01513742620504725, frozenset({'tunny

In [183]:
rules = generateRules(L, supportData, minConf=0.7) # 生成关联规则
print(rules) # 打印出关联规则

frozenset({' pasta ', 'brioches'})
frozenset({'water', ' pasta '})
frozenset({'water', 'biscuits'})
frozenset({'milk', 'yoghurt'})
frozenset({'frozen vegetables', 'milk'})
frozenset({'water', 'milk'})
frozenset({'milk', 'brioches'})
frozenset({' pasta ', 'yoghurt'})
frozenset({'milk', 'tomato souce'})
frozenset({' pasta ', 'tomato souce'})
frozenset({'milk', 'coffee'})
frozenset({' pasta ', 'coffee'})
frozenset({'biscuits', 'milk'})
frozenset({'biscuits', ' pasta '})
frozenset({' pasta ', 'milk'})
frozenset({'coke', 'milk'})
frozenset({'water', ' pasta ', 'milk'})
[]


In [177]:

C1 = createC1(D) # 创建候选项集
C1

[frozenset({' pasta '}),
 frozenset({'beer'}),
 frozenset({'biscuits'}),
 frozenset({'brioches'}),
 frozenset({'coffee'}),
 frozenset({'coke'}),
 frozenset({'crackers'}),
 frozenset({'frozen fish'}),
 frozenset({'frozen vegetables'}),
 frozenset({'ice cream'}),
 frozenset({'juices'}),
 frozenset({'milk'}),
 frozenset({'mozzarella'}),
 frozenset({'oil'}),
 frozenset({'rice'}),
 frozenset({'tomato souce'}),
 frozenset({'tunny'}),
 frozenset({'water'}),
 frozenset({'yoghurt'})]

In [169]:
L1, supportData0 = scanD(D, C1, 0.2) # 从候选项集中筛选出满足最小支持度的项集
print(L1) # 打印出满足最小支持度的项集

[frozenset({'biscuits'}), frozenset({'water'}), frozenset({' pasta '}), frozenset({'milk'})]


In [170]:
print(supportData0) # 打印出每个项集的支持度

{frozenset({'mozzarella'}): 0.01513742620504725, frozenset({'tunny'}): 0.10386436866120277, frozenset({'milk'}): 0.46132387604610425, frozenset({' pasta '}): 0.35034491706852927, frozenset({'coke'}): 0.10706485305884134, frozenset({'crackers'}): 0.030404601777566334, frozenset({'water'}): 0.27850701727829075, frozenset({'biscuits'}): 0.20474450187055337, frozenset({'juices'}): 0.08256384750124343, frozenset({'oil'}): 0.0356594511601756, frozenset({'coffee'}): 0.15027139242696191, frozenset({'tomato souce'}): 0.11597430962524058, frozenset({'yoghurt'}): 0.15234738230651126, frozenset({'brioches'}): 0.15319075319507816, frozenset({'beer'}): 0.1092057176221266, frozenset({'frozen vegetables'}): 0.11389831974569124, frozenset({'rice'}): 0.06502605799796726, frozenset({'frozen fish'}): 0.025279501762428908, frozenset({'ice cream'}): 0.01656466924723742}


In [171]:
L = [L1]
print(L)
print(len(L))
k = 2

while (len(L[k-2]) > 0): # 在L[k-2]中的项集的元素个数大于0时，继续循环
    Ck = aprioriGen(L[k-2], k)
    Lk, supK = scanD(D, Ck, 0.05)
    supportData.update(supK)
    L.append(Lk)
    k += 1 # k表示项集中元素的个数

# 去掉L中长度为1的
L = [L[i] for i in range(len(L)) if len(L[i]) > 1]

print(L) # 打印出所有满足最小支持度的项集
print (len(L)) # 打印出每个项集的支持度

[[frozenset({'biscuits'}), frozenset({'water'}), frozenset({' pasta '}), frozenset({'milk'})]]
1
[[frozenset({'biscuits'}), frozenset({'water'}), frozenset({' pasta '}), frozenset({'milk'})], [frozenset({'water', ' pasta '}), frozenset({'water', 'biscuits'}), frozenset({'water', 'milk'}), frozenset({'biscuits', 'milk'}), frozenset({'biscuits', ' pasta '}), frozenset({' pasta ', 'milk'})]]
2


In [172]:
print(supportData) # 打印出每个项集的支持度
print(len(supportData)) # 打印出项集的个数

{frozenset({' pasta ', 'milk'}): 0.1606513418247086, frozenset({'biscuits', ' pasta '}): 0.07763337153731376, frozenset({'biscuits', 'milk'}): 0.10550786064917933, frozenset({'water', 'milk'}): 0.13007374089051316, frozenset({'water', 'biscuits'}): 0.05912246177799883, frozenset({'water', ' pasta '}): 0.09551715935384815, frozenset({'biscuits', 'milk', ' pasta '}): 0.04491490603983306, frozenset({'water', ' pasta ', 'milk'}): 0.05347836429297407, frozenset({'water', ' pasta ', 'biscuits'}): 0.02662024522630452, frozenset({'water', 'biscuits', 'milk'}): 0.03561620137101831}
10


In [173]:
# 获得规则关联集合

rules = generateRules(L, supportData, minConf=0.1) # 生成关联规则
print(rules) # 打印出关联规则

frozenset({'water', ' pasta '})


KeyError: frozenset({' pasta '})

In [118]:
# test3

dataSet = loadDataSet()
dataSet.head()

dataSet.drop(['COD'], axis=1, inplace=True)
dataSet.head()

,pasta,milk,water,biscuits,coffee,brioches,yoghurt,frozen vegetables,tunny,beer,tomato souce,coke,rice,juices,crackers,oil,frozen fish,ice cream,mozzarella,tinned meat
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [126]:
L, supportData = apriori(dataSet) # 生成所有满足最小支持度的项集
print(L) # 打印出所有满足最小支持度的项集
print(supportData) # 打印出每个项集的支持度

[['tunny', 'mozzarella'], ['milk'], [' pasta ', 'milk', 'coke', 'crackers'], ['water'], ['tunny'], [' pasta ', 'milk', 'biscuits', 'crackers'], ['biscuits', 'milk', 'juices'], [' pasta '], ['oil'], ['biscuits'], [], [' pasta ', 'milk', 'coffee'], ['tunny', 'milk', 'tomato souce', 'juices', ' pasta '], ['milk'], [' pasta ', 'yoghurt'], ['biscuits', 'milk', 'crackers', 'brioches'], [' pasta '], [' pasta ', 'milk'], [' pasta ', 'juices', 'coffee'], ['juices', 'brioches'], ['biscuits'], [' pasta '], ['milk'], ['water', 'milk', 'coffee'], [' pasta '], ['coke', 'beer'], [' pasta ', 'milk'], ['biscuits', 'mozzarella'], ['frozen vegetables'], [' pasta ', 'milk'], ['rice'], [' pasta ', 'biscuits', 'crackers'], [' pasta ', 'milk', 'frozen vegetables'], ['milk', 'yoghurt', 'coffee'], ['water', 'rice', 'tomato souce'], ['milk'], ['biscuits'], ['tunny', 'milk', 'brioches'], [' pasta ', 'milk', 'tomato souce', 'crackers'], ['milk'], ['frozen vegetables'], ['biscuits'], ['milk'], ['biscuits', 'oil'],